In [1]:
# 우주선인가, 빙산인지를 결정하는 프로그램을 만들어보는 것.
from google.colab import drive
drive.mount('/content/drive',force_remount=True)



Mounted at /content/drive


In [2]:
import numpy as np # 선형대수 연산을 위함
import pandas as pd # 데이터 전처릴 위해서 사용. csv파일의 I/O

In [4]:
from subprocess import check_output
# 파이썬 프로그램내에서 새로운 프로세스 스폰하고 입출력 파이프를 연결하며 리턴코드를 획득할 수 있게 만드는 모듈.
print(check_output(["ls", "/content/drive/MyDrive/AI 고급과정/Project2/data/processed"]).decode("utf8"))

sample_submission.csv
test.json
train.json
TransferLearning.ipynb



In [5]:
#Mandatory imports 
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit
from os.path import join as opj
# 경로 지정
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import pylab
plt.rcParams['figure.figsize']=10,10
%matplotlib inline
#IPython에서 제공하는 Rich Output에 대한 표현방식, 즉, 도표와 같은 그림, 등을 notebook을 실행한 브라우저에서 바로 볼수있도록 도와주는 것.


In [6]:
train = pd.read_json("/content/drive/MyDrive/AI 고급과정/Project2/data/processed/train.json")
target_train=train['is_iceberg']
test = pd.read_json("/content/drive/MyDrive/AI 고급과정/Project2/data/processed/test.json")


ID : 이미지의 ID
band_1, band_2 ->  flattened image data, 75x75개의 픽셀값이 있으므로, 목록에는 5625개의 데이터가 존재한다.

inc_angle : 이미지가 촬영된 발생 각도이다. "na" 발생 각도가 있는 이미지는 모두 누출을 방지하기 위해서 훈련 데이터에 존재한다.

is_target : 대상 변수, 빙산일 경우 1로 설정되고, 선박일 경우 0으로 설정하였다. 이는 train_json에만 존재한다.


In [7]:
target_train = train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'],errors= 'coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'],errors='coerce')
# 데이터가 문자형인 부분을 강제로 Nan처리 해버리겠다는 얘기! 대신에 제외하고는 숫자형으로 변경된다! -> nan은 문자도, 아무것도 아닌값.

# 앞방향으로 채우면 'ffill 또는 pad' 뒷방향으로 채울때는 bfill 또는 backfill
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

# training data 만들어주기
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_1']])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])
X_band_3 = (X_band_1+X_band_2)/2
# VGG가 3개의 채널이 필요함을 기억하자.
# 3번째 채널을 구성하는 방법이 부족해서 평균화를 수행. 데이터 증강할 경우 도움이 된다고 생각합니다.
# HH와 HV Polarisation의 작동방식은 발생각도가 전체 반에 많은 영향을 미친다. 이렇게 구현한 방식에서는 입사각도 입력으로 가짐.
# 거의 발전이 없음. scipy.signal.fftconvolve(밴드_1, 밴드_1, 모드 = '동일) -> 해보기
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])

X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)


X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)



print(train.shape)
print(X_band_1.shape)

(1604, 5)
(1604, 75, 75)


test Data 만들어주기

In [13]:
#Import Keras.
from matplotlib import pyplot
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from tensorflow.keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU, PReLU
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	


In [14]:
# 다양한 인풋을 받아오기 위해서 
# from keras.proprocessing.image import ImageDataGenerator
batch_size= 64

# Define the Image transformation here
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip = True,
                         width_shift_range = 0,
                         height_shift_range=0,
                         channel_shift_range=0,
                         zoom_range=0.2,
                         rotation_range=10)



In [15]:
# 두개의 generator를 합치기 위해서다. 
# y 및 angle 배열 모두에 대해 동일한 랜덤 시드를 가진 정확히 동일한 제너레이터를 사용합니다.
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]
            # iterator 산출함수
# generator 는 간단하게 설명하면 iterator 를 생성해 주는 function 이다.
# generator 함수는 함수가 처음부터 시작되는게 아니라 yield 이후의 구문부터 시작하게 한다.


In [16]:
# Create Generator
def get_callbacks(filepath, patience=2):
  es = EarlyStopping('val_loss',patience=10, mode = 'min')
  msave = ModelCheckpoint(filepath, save_best_only=True)
  return [es,msave]

In [18]:

target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce') #inc_angle에서 na 값을 NAN으로 바꿔줌
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#133개의 NA가 있음
train['inc_angle']=train['inc_angle'].fillna(method='pad') #결측값을 앞 방향부터 채워나감
X_angle1=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

# training data를 생성합니다.
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2 #band_1과 band_2 각 행렬의 값을 더한 평균값으로 만듦. 그래서 band_1과 bnad_2의 크기가 같아야됨.
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])

#X_band_1[:, :, :, np.newaxis] -> 같은 배열에 대해 차원을 1차원 증가
# axis=-1 설정 -> 맨 마지막 차원 내에서 합쳐짐
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)

print(X_train.shape)

X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)
print(X_test)
#Import Keras.
from matplotlib import pyplot
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from tensorflow.keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU, PReLU
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping



from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64
# image transformations 정의
gen = ImageDataGenerator(horizontal_flip = True, #수평 방향으로 뒤집기
                         vertical_flip = True, #수직 방향으로 뒤집기
                         width_shift_range = 0., # 지정된 수평방향 이동 범위 내에서 임의로 원본이미지를 이동
                         height_shift_range = 0., # 지정된 수직방향 이동 범위 내에서 임의로 원본이미지를 이동
                         channel_shift_range=0, # 임의 채널 이동을 위한 범위
                         zoom_range = 0.2, # 지정된 확대/축소 범위내에서 임의로 원본이미지를 확대/축소 -> 0.8배에서 1.2배 크기 변화 시킴
                         rotation_range = 10) # 지정된 각도 범위내에서 임의로 원본이미지를 회전

# 두 generators를 합치는 함수입니다
# y와 angle array 모두에 동일한 랜덤 시드의 같은 제네레이터를 사용해야합니다
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #array는 동일합니다 - 마음은 안정되었으나 training 속도가 느려졌습니다.
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

#  generator를 생성합니다
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min") # 더 이상 개선의 여지가 없으면 학습 조기 종료
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2) # 출력 뉴런의 수는 1
    base_model = VGG16(weights='imagenet', include_top=False, # weights="imagenet" : pre-training on ImageNet
                 input_shape=X_train.shape[1:], classes=1) 
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one) # Dropout : 일부 weight만 사용
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

# 데이터 확장과 함께 K-fold Cross Validation을 사용합니다.
def myAngleCV(X_train,X_angle,X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        # file path와 callbacks를 정의합니다.
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        # 최적의 모델을 가져옵니다.
        galaxyModel.load_weights(filepath=file_path)
        # Training score를 가져옵니다.
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        # Test Score를 가져옵니다.
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        # validation Score를 가져옵니다.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        # Test Scores를 가져옵니다.
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        # Train Scores를 가져옵니다.
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

(1604, 75, 75, 3)
[[[[-15.863251 -21.629612 -18.746431]
   [-15.201077 -21.142353 -18.171715]
   [-17.887735 -23.908337 -20.898037]
   ...
   [-14.146514 -23.479631 -18.813072]
   [-12.898778 -24.123251 -18.511015]
   [-14.006273 -26.40205  -20.204163]]

  [[-16.395206 -20.985918 -18.690563]
   [-15.201077 -21.142353 -18.171715]
   [-16.395063 -26.701574 -21.548319]
   ...
   [-15.438498 -26.696655 -21.067577]
   [-14.006346 -23.075493 -18.54092 ]
   [-13.73268  -25.574196 -19.653439]]

  [[-15.443562 -23.90848  -19.676022]
   [-14.365982 -23.908407 -19.137196]
   [-17.059881 -25.579258 -21.319569]
   ...
   [-17.054962 -24.123323 -20.589142]
   [-15.35687  -23.075493 -19.21618 ]
   [-13.800274 -24.348803 -19.07454 ]]

  ...

  [[-14.586817 -25.580256 -20.083536]
   [-17.160078 -23.909264 -20.534672]
   [-15.863964 -24.586357 -20.22516 ]
   ...
   [-15.9454   -23.480413 -19.712906]
   [-18.798677 -21.793922 -20.296299]
   [-19.821657 -23.275888 -21.548773]]

  [[-15.044075 -26.123302 -

In [19]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0
58900480/58889256 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:146: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/100
24/24 [==============================] - 26s 503ms/step - loss: 0.6939 - accuracy: 0.5900 - val_loss: 0.4081 - val_accuracy: 0.7925
Epoch 2/100
24/24 [==============================] - 8s 350ms/step - loss: 0.3887 - accuracy: 0.8220 - val_loss: 0.3450 - val_accuracy: 0.8224
Epoch 3/100
24/24 [==============================] - 8s 351ms/step - loss: 0.3129 - accuracy: 0.8531 - val_loss: 0.2699 - val_accuracy: 0.8710
Epoch 4/100
24/24 [==============================] - 9s 367ms/step - loss: 0.2711 - accuracy: 0.8800 - val_loss: 0.2731 - val_accuracy: 0.8785
Epoch 5/100
24/24 [==============================] - 8s 348ms/step - loss: 0.2637 - accuracy: 0.8852 - val_loss: 0.2875 - val_accuracy: 0.8804
Epoch 6/100
24/24 [==============================] - 8s 330ms/step - loss: 0.2478 - accuracy: 0.8912 - val_loss: 0.2865 - val_accuracy: 0.8860
Epoch 7/100
24/24 [==============================] - 8s 347ms/step - loss: 0.2551 - accuracy: 0.8939 - val_loss: 0.2413 - val_accuracy: 0.904

In [20]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)